In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

## Loading Data

In [ ]:
master_data=[]
for i in ["train", "test"]:
    splity=[]
    p1="/Users/pranavthakkar/Desktop/srip/UCI HAR Dataset/"+i+"/Inertial Signals"
    # print(p1)
    for s_type in ['body_acc', 'body_gyro', 'total_acc']:
        p2=p1+"/"+s_type
        # print(p2)
        for axis in ['x', 'y', 'z']:
            file_path = p2 + f"_{axis}_{i}.txt"
            # print(file_path)
            d=np.loadtxt(file_path).reshape(-1, 128, 1)
            # print(d.shape)
            splity.append(d)
    split_data=np.concatenate(splity, axis=2)
    master_data.append(split_data)

X_train=master_data[0]
X_test=master_data[1]            



In [3]:
X_train.shape, X_test.shape

((7352, 128, 9), (2947, 128, 9))

## Loading labels data

In [ ]:
y_test = np.loadtxt("/Users/pranavthakkar/Desktop/srip/UCI HAR Dataset/test/y_test.txt").astype(int) 
y_train = np.loadtxt("/Users/pranavthakkar/Desktop/srip/UCI HAR Dataset/train/y_train.txt").astype(int) 

In [8]:
y_train.shape, y_test.shape

((7352,), (2947,))

## 1D CNN

In [46]:
m_cnn = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(128, 9)),  
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(), #for converting 2D to 1D for dense layer.
    Dense(64, activation='relu'),
    # Dense(24, activation='relu'),
    Dense(6, activation='softmax')  # 6 classes
])
m_cnn.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
m_cnn.summary()

/Users/pranavthakkar/Desktop/srip/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_27 (Conv1D)              │ (None, 126, 64)        │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_27 (MaxPooling1D) │ (None, 63, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_28 (Conv1D)              │ (None, 61, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_28 (MaxPooling1D) │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 3840)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │       245,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 272,710 (1.04 MB)

 Trainable params: 272,710 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
training_cnn = m_cnn.fit(X_train, y_train, epochs=20, batch_size=32,validation_data=(X_test, y_test))

Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7025 - loss: 0.7076 - val_accuracy: 0.8850 - val_loss: 0.3370
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9423 - loss: 0.1489 - val_accuracy: 0.8907 - val_loss: 0.3638
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9467 - loss: 0.1286 - val_accuracy: 0.8955 - val_loss: 0.3998
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9485 - loss: 0.1165 - val_accuracy: 0.9264 - val_loss: 0.2965
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9544 - loss: 0.1078 - val_accuracy: 0.9074 - val_loss: 0.3421
Epoch 6/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9563 - loss: 0.1004 - val_accuracy: 0.9118 - val_loss: 0.3714
Epoch 7/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9554 - loss: 0.1104 - val_accuracy: 0.9152 - val_loss: 0.3976
Epoch 8/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9555 - loss: 0.0948 - val_accuracy: 0.

In [48]:
print("Train Accuracy:", training_cnn.history['accuracy'][-1])
print("Test Accuracy:", training_cnn.history['val_accuracy'][-1])

Train Accuracy: 0.9753808379173279
Test Accuracy: 0.9219545125961304


## LSTM

In [51]:
m_lstm = Sequential([
    LSTM(84, return_sequences=True, input_shape=(128, 9)),  # First LSTM layer
    LSTM(42),  # Second LSTM layer
    Dense(18, activation='relu'),
    Dense(6, activation='softmax')  # Output layer for 6 classes
])
m_lstm.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
m_lstm.summary()

/Users/pranavthakkar/Desktop/srip/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128, 84)        │        31,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 42)             │        21,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 18)             │           774 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 6)              │           114 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,808 (210.19 KB)

 Trainable params: 53,808 (210.19 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# Train LSTM Model
training_lstm = m_lstm.fit(X_train, y_train, epochs=20, batch_size=32,validation_data=(X_test, y_test))

Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.4421 - loss: 1.2741 - val_accuracy: 0.5813 - val_loss: 0.8791
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.7102 - loss: 0.6934 - val_accuracy: 0.6810 - val_loss: 0.7173
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.7905 - loss: 0.4994 - val_accuracy: 0.8035 - val_loss: 0.5168
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.8644 - loss: 0.3482 - val_accuracy: 0.8992 - val_loss: 0.2970
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.9274 - loss: 0.2137 - val_accuracy: 0.8911 - val_loss: 0.3726
Epoch 6/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 13s 57ms/step - accuracy: 0.9332 - loss: 0.1713 - val_accuracy: 0.9030 - val_loss: 0.3292
Epoch 7/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 13s 58ms/step - accuracy: 0.9446 - loss: 0.1356 - val_accuracy: 0.9006 - val_loss: 0.3212
Epoch 8/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 13s 55ms/step - accuracy: 0.9242 - loss: 0.1871 - 

In [53]:
print("Train Accuracy:", training_lstm.history['accuracy'][-1])
print("Test Accuracy:", training_lstm.history['val_accuracy'][-1])

Train Accuracy: 0.9507616758346558
Test Accuracy: 0.9012554883956909


## Summary

We trained 2 Deep Learning models : 1D-CNN and LSTM on the raw acceleeration data.
And the 1D-CNN performed better in both Training and test accuracy. Also, 1D-CNN took much lesser time compared to LSTM.
